# Introduction
The objective of this introductory notebook is to learn how to read a data file and obtain some first insights. 

#### Required libraries
* <a href = "https://pandas.pydata.org/"><code>Pandas</code></a>
* <a href = "https://www.scipy.org/"><code>Scipy</code></a> 
* <a href = "https://numpy.org/"><code>Numpy</code></a> 

#### Table of contents
<ol>
    <li><a href="#read_data_files">Read data files</a></li>
    <li><a href="#save_data_files">Save data files</a></li>
    <li><a href="#subsetting_dataframe">Subsetting a DataFrame</a><br>
        3.1. <a href="#subset_selection_indexing_operator">Subset selection with <code>[]</code></a><br>
        3.2. <a href="#subset_selection_loc">Subset selection with <code>.loc</code></a><br>
        3.3. <a href="#subset_selection_iloc">Subset selection with <code>.iloc</code></a>
    </li>
    <li><a href="#insights_dataframe">Getting insights from a DataFrame</a><br>
        4.1. <a href="#data_types">Data types</a><br>
        4.2. <a href="#describe"><code>describe</code></a>
    </li>
</ol>

***

<h2 id="read_data_files">1 - Read data files</h2>

There exist several file formats for storing data. Among the most used we find:
* Comma separated values (**CSV**).
* Attribute-Relation File Format (**ARFF**).
* Microsoft Excel (XLSX).
* JavaScript Object Notation (JSON).

The <a href = "https://pandas.pydata.org/">**Pandas**</a> library is a very useful tool with which we can read these and other file formats. When a data file is readed, it is then loaded in memory using a specific data structure called <code>DataFrame</code>, which allow us to easily work with its internal data. A <code>DataFrame</code> is two-dimensional size-mutable, potentially heterogeneous tabular data structure with labeled axes (rows and columns).

In [ ]:
import pandas as pd

#dir(pd)
#dir(pd.DataFrame)
#help(pd.DataFrame.shape)

To better understand the inner workings of this library, we are going to work with the <a href = "https://archive.ics.uci.edu/ml/datasets/automobile">**Automobile**</a> dataset which is open-source and available in the <a href = "https://archive.ics.uci.edu/ml/index.php">**UCI Machine learning repository**</a>. Although we can download the dataset from the official website, it is in its own format (<code>.data</code>), for this reason and for simplicity we will work with our own copy.
* <a href = "https://raw.githubusercontent.com/ferjorosa/telefonica-talentum-ai-2019/master/Machine%20learning/regression/Automobile_data.csv">**CSV version**</a>
* <a href = "https://raw.githubusercontent.com/ferjorosa/telefonica-talentum-ai-2019/master/Machine%20learning/regression/Automobile_data.arff">**ARFF version**</a>

We use <code>pandas.read_csv()</code> function to read the csv file. In the bracket, we put the file path along with a quotation mark, so that pandas will read the file into a data frame from that address. The file path can be either an URL or your local file address.<br>

In [ ]:
#df = pd.read_csv("https://raw.githubusercontent.com/ferjorosa/ai-introduction-2019/master/Machine%20learning/introduction/Data/Automobile_clean.csv")                               # From URL
df = pd.read_csv("../Data/Automobile_clean.csv")   # From disk

After reading the dataset, we can use the <code>dataframe.head(n)</code> method to check the top n rows of the dataframe; where n is an integer. Contrary to <code>dataframe.head(n)</code>, <code>dataframe.tail(n)</code> will show you the bottom n rows of the dataframe.

In [ ]:
# show the first 5 rows using dataframe.head() method
print("The first 5 rows of the dataframe") 
df.head(5)

<h2 id="save_data_files">2 - Save data files</h2>
<p>
Correspondingly, Pandas enables us to save the dataset to csv  by using the <code>dataframe.to_csv()</code> method, you can add the file path and name along with quotation marks in the brackets.
</p>
<p>
    For example, if we would save the dataframe <code>df</code> as <b>automobile.csv</b> to our local machine, we may use the syntax below:
</p>

In [ ]:
df.to_csv("automobile.csv", index=False) 

# Extra for Google Colab users:
# from google.colab import files
# files.download("automobile.csv")

We can read or load in other formats similar to how we do it with CSV files:

| Data format  | Read           | Save             |
| ------------- |:--------------:| ----------------:|
| csv           | `pd.read_csv()`  |`df.to_csv()`     |
| json          | `pd.read_json()` |`df.to_json()`    |
| excel         | `pd.read_excel()`|`df.to_excel()`   |
| hdf           | `pd.read_hdf()`  |`df.to_hdf()`     |
| sql           | `pd.read_sql()`  |`df.to_sql()`     |
| ...           |   ...          |       ...        |


However, we will notice the absence of the <a href="https://www.cs.waikato.ac.nz/ml/weka/arff.html">**ARFF data type**</a>. This is because it is not directly supported by Pandas. In order to load an ARFF file we have to use Scipy. It will first generate a Numpy's <code>Ndarray</code> and then we will pass that array to the <code>DataFrame</code>'s constructor.

In [ ]:
from scipy.io import arff

# Scipy no permite cargar arhivos Arff desde una URL directamente como sí nos permite Pandas
data_arff = arff.loadarff("../Data/Automobile_data.arff")
df_arff = pd.DataFrame(data_arff[0])

df_arff.head()

In addition, in order to load an ARFF dataSet from an URL we have to do some extra work if we want to load it into Pandas (in comparison with a CSV file).

(I got this solution from the following <a href="https://stackoverflow.com/questions/53951017/how-can-i-get-arff-loadarff-to-work-with-urllib-request-to-read-a-arff-file-from">**source**</a>)

In [ ]:
from scipy.io import arff
import urllib.request
import io # for io.StringIO()

url = "https://raw.githubusercontent.com/ferjorosa/telefonica-talentum-ai-2019/master/Machine%20learning/introduction/Data/Automobile_data.arff"
ftpstream = urllib.request.urlopen(url)
data_arff = arff.loadarff(io.StringIO(ftpstream.read().decode('utf-8')))
df_arff = pd.DataFrame(data_arff[0])

df_arff.head()

We will notice that all the columns with discrete values have a <code>b'</code> preceding each row value. This is because Scipy has loaded each of these values as a sequence of bytes. We can fix this by transforming them into strings:

In [ ]:
import numpy as np

#float_df = df_arff.select_dtypes(float) # To test how to select other columns by their data type

str_df = df_arff.select_dtypes([np.object])
str_df = str_df.stack().str.decode('utf-8').unstack()
str_df.head(5)

<h2 id="subsetting_dataframe">3 - Subsetting a DataFrame </h2>
A DataFrame is composed of three different components, the index, columns, and the data. The data is also known as the values. The main takeaway from the DataFrame anatomy is that each row has a label and each column has a label. These labels are used to refer to specific rows or columns in the DataFrame. It’s the same as how humans use names to refer to specific people.

In [ ]:
index = df.index
columns = df.columns
values = df.values

print(type(index))
print(type(columns))
print(type(values))

Subsetting is simply selecting particular rows and columns of data from a DataFrame (or Series). This could mean selecting all the rows and some of the columns, some of the rows and all of the columns, or some of each of the rows and columns.

There are 3 main ways to do subset selection with a DataFrame in Pandas:
* <code>[]</code> operator (also referred as the **indexing operator**). 
* <code>loc</code> attribute followed by the <code>[]</code> operator. 
* <code>iloc</code> attribute followed by the <code>[]</code> operator.

<h3 id="subset_selection_indexing_operator">3.1 - Subset selection with <code>[]</code></h3>
Its primary purpose is to select columns by the column names. We can select one or multiple columns.

In [ ]:
df["make"].head(5)

Selecting a single column of data returns the other pandas data container, the <code>Series</code>. A <code>Series</code> is a one-dimensional sequence of labeled data. There are two main components of a Series, the index and the data(or values). There are NO columns in a Series.

In [ ]:
df[["normalized-losses"]].head(5)

Using the double <code>[]</code> operator we get a <code>DataFrame</code> instead of a <code>Series</code> object.

In [ ]:
df[["fuel-system", "make"]].tail(5) # we dont have to follow the original column order when subsetting

<h3 id="subset_selection_loc">3.2 - Subset selection with <code>.loc</code></h3>

The .loc indexer selects data in a different way than just the indexing operator. It can select subsets of rows or columns. It can also simultaneously select subsets of rows and columns. Most importantly, it only selects data by the **label** of the rows and columns.

<span style="color:red">**Note:** If no labels have been assigned we have to use the index (row or column)</span>

#### Example without labels on columns or rows

In [ ]:
df = pd.DataFrame([[1, 2], [4, 5], [7, 8]])
df

In [ ]:
df.loc[0,0]

#### Example with labels on both columns and rows

In [ ]:
df = pd.DataFrame([[1, 2], [4, 5], [7, 8]],
                  index=['cobra', 'viper', 'sidewinder'],
                  columns=['max_speed', 'shield'])
df

In [ ]:
df.loc["viper"]

In [ ]:
df.loc["viper", "max_speed"]

#### Example with column labels but no labels on the rows

In [ ]:
df.loc[0] # first row, returns a Series

It is possible to ‘slice’ the rows of a DataFrame with .loc by using slice notation. Slice notation uses a colon to separate start, stop and step values. For instance we can select all the rows from Niko through Dean like this:

In [ ]:
df.loc[0:5]

In [ ]:
df.loc[5::2] # Slice from 5 to the end with a step of 2 

In [ ]:
df.loc[3,"make"] # Select using the row and column indexes

----

**Question: Using the slice notation, select all the rows (except the first 5 ones) of the 'symboling' and 'make' columns** 

(double click for solution)
    
<!--
df.loc[5:, ["symboling", "make"]]
-->

----

In [ ]:
#### Put your code here


<h3 id="subset_selection_iloc">3.2 - Subset selection with <code>.iloc</code></h3>
The <code>.iloc</code> indexer is very similar to .loc but only uses integer locations to make its selections. The word <code>.iloc</code> itself stands for integer location so that should help with remember what it does.

Given that the row index is an integer, its syntax for this example is identical in many cases:

In [ ]:
df.iloc[0:5] # Select all columns for rows 0 to 4 (notice the difference with .loc for this case)

In [ ]:
df.iloc[5::2, 0] # Slice from 5 to the end with a step of 2 only for the first column

<h2 id="insights_dataframe">4 - Getting insights from a DataFrame </h2>

There are several ways to obtain essential insights of the data to help us better understand our dataset.


<span style="color:red">**Note:** For more information about descriptive statistics, take a look into module 3 (exploratory data analysis).</span>

<h3 id ="data_types">4.1 - Data types</h3>
The main types stored in Pandas dataframes are <code>object</code>, <code>float</code>, <code>int</code>, <code>bool</code> and <code>datetime64</code>. In order to better learn about each attribute, it is always good for us to know the data type of each column. In Pandas:

In [ ]:
df.dtypes

returns a Series with the data type of each column.
<p>
As a result, as shown above, it is clear to see that the data type of "symboling" and "curb-weight" are <code>int64</code>, "normalized-losses" is <code>object</code>, and "wheel-base" is <code>float64</code>, etc.
</p>
<p>
These data types can be changed; we will learn how to accomplish this later.
</p>

<h3 id="describe">4.2 - <code>describe</code></h3>
If we would like to get a statistical summary of each column, such as count, column mean value, column standard deviation, etc. We use the describe method:

In [ ]:
df.describe()

This shows the statistical summary of all numeric-typed (int, float) columns.<br>

For example, the attribute "symboling" has 205 counts, the mean value of this column is 0.83, the standard deviation is 1.25, the minimum value is -2, 25th percentile is 0, 50th percentile is 1, 75th percentile is 2, and the maximum value is 3.

However, what if we would also like to check all the columns including those that are of type object? In that case we can add the argument <code>include = "all"</code> inside the bracket:

In [ ]:
# describe all the columns in "df" 
df.describe(include = "all")

Now, it provides the statistical summary of all the columns, including object-typed attributes. We can now see how many unique values, which is the top value and the frequency of top value in the object-typed columns.

Some values in the table above show as "NaN", this is because those numbers are not available regarding a particular column type.

Missing values are not being considered.

----

**Question: Describe both the 'length' and 'compression-ratio' columns**

(double-click for solution)

<!--
df[["length", "compression-ratio"]].describe()
-->

----

In [ ]:
#### Put your code here


## References

#### Links
1. <a href="https://www.coursera.org/learn/data-analysis-with-python/">Santarcangelo, J. (2019). "Data analysis with Python".</a>
1. <a href="https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-6fcd0170be9c">Petrou, T. (2017). "Selecting Subsets of Data in Pandas: Part 1".</a>